# Laboratorul 3: Satisfacerea parțială a restricțiilor
 - Tudor Berariu
 - Andrei Olaru

In [1]:
from copy import copy, deepcopy
from itertools import combinations

## O problemă de satisfacere a restricțiilor

O **problemă de satisfacere a restricțiilor** este definită de:
 - o mulțime discretă de **variabile**
 - câte un **domeniu de valori** pentru fiecare variabilă
 - un set de **constrângeri** impuse asupra unor grupuri de variabile
 
Vom reprezenta în Python cele de mai sus astfel:
 - fiecare variabilă va fi reprezentată printr-un șir de caractere

```
Vars = ["A", "B", "C"]
```
 - mulțimea domeniilor va fi un dicționar având câte o intrare pentru fiecare variabilă:
    + cheie va fi numele variabilei
    + valoare va fi domeniul de valori al acelei variabile

```
Domains = {"A": [1, 2, 3], "B": [1, 5, 9], "C": [-2, -1]}
```
 - o constrângere va fi reprezentată printr-un tuplu format din:
    + lista de variabile implicată în constrângere
    + o funcție anonimă care întoarce `True` sau `False`

```
Constraints = [(["A", "B", "C"], lambda a, b, c: a + b + c == 0)]
```

Vom reprezenta o **soluție** printr-un dicționar ce indică o valoare pentru fiecare variabilă (e.g. `{"A": 1, "B": 1, "C" -2}`) și vom defini **costul** ca fiind egal cu numărul de constrângeri încălcate de acea soluție.

In [2]:
VarsA = ["A", "B", "C", "D", "E"]
DomainsA = {v: [i for i in range(10)] for v in VarsA}
ConstraintsA = [(list(p), lambda x,y: x != y) for p in combinations(VarsA, 2)] # toate valorile diferite
ConstraintsA.append((["A","B"], lambda a, b: a + b == 10))
ConstraintsA.append((["B","D"], lambda b, d: b + d == 6))
ConstraintsA.append((["C"], lambda c: c < 5))
ConstraintsA.append((["A"], lambda a: a > 5))
ConstraintsA.append((["A","B","C","D","E"], lambda a, b, c, d, e: a + b + c + d + e == 30))
MathProblem = {"Vars": VarsA, "Domains": DomainsA, "Constraints": ConstraintsA}

In [3]:
VarsC = ["France", "Germany", "Loux", "Belgium", "Netherlands"]
DomainsC = {v: ["blue", "red", "yellow", "green"] for v in VarsC}
ConstraintsC = []
for (a, b) in [("France", "Germany"), ("France", "Belgium"), ("France", "Loux"),
               ("Belgium", "Netherlands"), ("Belgium", "Loux"), ("Belgium", "Germany"),
               ("Loux", "Germany"), ("Netherlands", "Germany")]:
    ConstraintsC.append(([a, b], lambda a, b: a != b))
ColoringProblem = {"Vars": VarsC, "Domains": DomainsC, "Constraints": ConstraintsC}

In [4]:
Nr_A = 2
Nr_B = 2
Nr_C = 1
Nr_D = 2
Nr_total = Nr_A + Nr_B + Nr_C + Nr_D

VarsCar=["Car_" + str(i+1) for i in range(Nr_total)]
DomainsCar={v: ["TypeA", "TypeB", "TypeC", "TypeD"] for v in VarsCar}

CarSetup = {
    "TypeA" : ["AC", "PowerBrakes", "Radio"],
    "TypeB" : ["Sunroof", "AC"],
    "TypeC" : ["Sunroof", "Radio", "PowerBrakes"],
    "TypeD" : ["Radio", "AC"]
}

def car_type_constraint(*car_vars):
    nr_typeA = len(list(filter(lambda x: x == "TypeA", car_vars)))
    nr_typeB = len(list(filter(lambda x: x == "TypeB", car_vars)))
    nr_typeC = len(list(filter(lambda x: x == "TypeC", car_vars)))
    nr_typeD = len(list(filter(lambda x: x == "TypeD", car_vars)))
    
    return nr_typeA == Nr_A and nr_typeB == Nr_B and nr_typeC == Nr_C and nr_typeD == Nr_D 

def sunroof_workarea_constraint(*car_vars):
    NR_MAX = 3
    ct = 0
    for v in car_vars:
        if "Sunroof" in CarSetup[v]:
            ct += 1
    
    return ct <= NR_MAX

def radio_workarea_constraint(*car_vars):
    NR_MAX = 2
    ct = 0
    for v in car_vars:
        if "Radio" in CarSetup[v]:
            ct += 1
    
    return ct <= NR_MAX

ConstraintsCar = []
ConstraintsCar.append(([v for v in VarsCar], car_type_constraint))

for i in range(Nr_total - 4):
    ConstraintsCar.append((VarsCar[i:(i+5)], sunroof_workarea_constraint))

for i in range(Nr_total - 2):
    ConstraintsCar.append((VarsCar[i:(i+3)], radio_workarea_constraint))
    
CarProblem = {"Vars": VarsCar, "Domains": DomainsCar, "Constraints": ConstraintsCar}

## Cerința 1

Implementați funcția `get_constraints` care primește o variabilă `var` și o listă de constrângeri `constraints` și întoarce doar acele constrângeri care implică variabila dată.
```
Constraints = [(["A", "B"], lambda a,b: a<b), (["A"], lambda a: a<5)]
get_constraints("B", Constraints)
==> [(["A", "B"], lambda a,b: a<b)]
get_constraints("A", Constraints)
==> [(["A", "B"], lambda a,b: a<b), (["A"], lambda a: a<5)]
```

In [5]:
def get_constraints(var, constraints):
    return [constr for constr in constraints if var in constr[0]]

get_constraints("France", ConstraintsC) # => [(['France', 'Germany'], ...), (['France', 'Belgium'], ...), (['France', 'Loux'], ...)]

[(['France', 'Germany'], <function __main__.<lambda>(a, b)>),
 (['France', 'Belgium'], <function __main__.<lambda>(a, b)>),
 (['France', 'Loux'], <function __main__.<lambda>(a, b)>)]

## Cerința 2

Implementați funcția `fixed_constraints` care primește o soluție parțială `solution` și un set de constrângeri `constraints` și întoarce doar acele constrângeri care pot fi evaluate (i.e. variabilele implicate se regăsesc în soluția parțială).

In [6]:
def fixed_constraints(solution, constraints):
    return [constrs for constrs in constraints if all(constr in solution.keys() for constr in constrs[0])]

print(fixed_constraints({"France": "blue", "Belgium": "green"}, ConstraintsC)) # => [(['France', 'Belgium'], ...)]
print(fixed_constraints({"A": "1", "C": "2"}, ConstraintsA)) # => [(['A', 'C'], ...), (['C'], ...), (['A'], ...)]

[(['France', 'Belgium'], <function <lambda> at 0x7f7328abe9d0>)]
[(['A', 'C'], <function <listcomp>.<lambda> at 0x7f7328abe040>), (['C'], <function <lambda> at 0x7f7328abe5e0>), (['A'], <function <lambda> at 0x7f7328abe670>)]


## Cerința 3

Implementați funcția `check_constraint` care primește o constrângere `constraint` și o soluție parțială `solution` și întoarce `True` dacă soluția respectă constrângerea și `False` altfel.

**Hint:** pentru a apela o funcție știind lista parametrilor săi folosiți `f(*args)`, unde `args` este lista de argumente.

In [7]:
def check_constraint(solution, constraint):
    return constraint[1](*[solution[var] for var in constraint[0]])

print(check_constraint({"France": "blue", "Belgium": "green"}, ConstraintsC[1])) # => True
print(check_constraint({"France": "blue", "Belgium": "blue"}, ConstraintsC[1])) # => False
print(check_constraint({"C": 10, "A": 10}, ConstraintsA[-2])) # => True
print(check_constraint({"C": 10, "A": 3}, ConstraintsA[-2])) # => False

True
False
True
False


## Cerința 4: PCSP

Completați ceea ce lipsește în algoritmul PCSP.

* `vars` -- variabilele care mai rămân de verificat
* `domains` -- domeniile pentru variabilele care mai rămân de verificat, cu valorile care mai rămân de verificat pentru fiecare variabilă
* `constrains` -- lista de constrângeri
* `acceptable_cost` -- costul pentru care se consideră ca soluția este satisfăcătoare
* `solution` -- soluția parțială, conținând valori pentru variabilele verificate până acum
* `cost` -- costul soluției parțiale (`solution`) -- numărul de constrângeri nesatisfăcute

Valoare întoarsă de funcție este `True` dacă a fost găsită o soluție completă satisfăcătoare (vezi costul acceptabil), și `False` altfel.

Se folosesc două variabile globale:
* `best_cost` -- cel mai bun (mic) cost întâlnit până acum în explorare, pentru o soluție completă
* `best_solution` -- soluția corespunzătoare celui mai bun cost

In [8]:
def PCSP(vars, domains, constraints, acceptable_cost, solution, cost):
    global best_solution
    global best_cost

    if not vars:
        # Dacă nu mai sunt variabile, am ajuns la o soluție mai bună
        print(f"New best: {cost} - {solution}")

        # Salvați soluția nou-descoperită
        best_cost = cost
        best_solution = solution

        # Dacă este suficient de bună, funcția întoarce True
        return cost <= acceptable_cost

    elif not domains[vars[0]]:
        # Dacă nu mai sunt valori în domeniu, am terminat căutarea
        return False
    elif cost >= best_cost:
        # Dacă am ajuns deja la un cost identic cu cel al celei mai bune soluții, nu mergem mai departe
        return False
    else:
        # Luăm prima variabilă și prima valoare din domeniu
        var = vars[0]
        val = domains[var].pop(0)

        # Construim noua soluție
        new_solution = {var: val}
        new_solution.update(solution)

        # Obținem lista constrângerilor ce pot fi evaluate acum
        new_constraints = fixed_constraints(new_solution, get_constraints(var, constraints))
        # Calculăm costul noii soluții parțiale (fiecare constrângere încălcată = 1)
        new_cost = cost + len(list(filter(lambda constr: not check_constraint(new_solution, constr), new_constraints)))

        # Verificăm dacă noul cost este mai mic decât cel mai bun cost
        if new_cost < best_cost:
            # Dacă noul cost este mai mic decât cel mai bun cunoscut, rezolvăm pentru restul variabilelor
            # Dacă apelul recursiv întoarce True, a fost găsită o soluție suficient de bună, deci întoarcem True
            new_domains = deepcopy(domains)
            new_domains.pop(var)

            if PCSP(vars[1:], new_domains, constraints, acceptable_cost, new_solution, new_cost):
                return True

        # Verificăm pentru restul valorilor
        return PCSP(vars, domains, constraints, acceptable_cost, solution, cost)

# Un wrapper care să instanțieze variabilele globale
def run_pcsp(problem, acceptable_cost):
    global best_solution
    global best_cost

    [vars, domains, constraints] = [problem[e] for e in ["Vars", "Domains", "Constraints"]]

    best_solution = {}
    best_cost = len(constraints)

    if PCSP(vars, deepcopy(domains), constraints, acceptable_cost, {}, 0):
        print(f"Best found: {best_cost} - {best_solution}")
    else:
        print(f"Acceptable solution not found; Best found: {best_cost} - {best_solution}")

# Rulăm măgăria
run_pcsp(MathProblem, 1)
run_pcsp(ColoringProblem, 1)
run_pcsp(CarProblem, 1)

New best: 14 - {'E': 0, 'D': 0, 'C': 0, 'B': 0, 'A': 0}
New best: 10 - {'E': 1, 'D': 0, 'C': 0, 'B': 0, 'A': 0}
New best: 8 - {'E': 1, 'D': 1, 'C': 0, 'B': 0, 'A': 0}
New best: 7 - {'E': 2, 'D': 1, 'C': 0, 'B': 0, 'A': 0}
New best: 6 - {'E': 1, 'D': 6, 'C': 0, 'B': 0, 'A': 0}
New best: 5 - {'E': 3, 'D': 2, 'C': 1, 'B': 0, 'A': 0}
New best: 4 - {'E': 2, 'D': 6, 'C': 1, 'B': 0, 'A': 0}
New best: 3 - {'E': 3, 'D': 5, 'C': 2, 'B': 1, 'A': 0}
New best: 2 - {'E': 9, 'D': 7, 'C': 4, 'B': 8, 'A': 2}
New best: 1 - {'E': 1, 'D': 2, 'C': 0, 'B': 4, 'A': 6}
Best found: 1 - {'E': 1, 'D': 2, 'C': 0, 'B': 4, 'A': 6}
New best: 6 - {'Netherlands': 'red', 'Belgium': 'blue', 'Loux': 'blue', 'Germany': 'blue', 'France': 'blue'}
New best: 4 - {'Netherlands': 'blue', 'Belgium': 'red', 'Loux': 'blue', 'Germany': 'blue', 'France': 'blue'}
New best: 3 - {'Netherlands': 'yellow', 'Belgium': 'red', 'Loux': 'blue', 'Germany': 'blue', 'France': 'blue'}
New best: 2 - {'Netherlands': 'yellow', 'Belgium': 'red', 'Lou

Expected for numbers:

```
New best: 14 - {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}
New best: 10 - {'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 1}
New best: 8 - {'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}
New best: 7 - {'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 2}
New best: 6 - {'A': 0, 'B': 0, 'C': 0, 'D': 6, 'E': 1}
New best: 5 - {'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 3}
New best: 4 - {'A': 0, 'B': 0, 'C': 1, 'D': 6, 'E': 2}
New best: 3 - {'A': 0, 'B': 1, 'C': 2, 'D': 5, 'E': 3}
New best: 2 - {'A': 2, 'B': 8, 'C': 4, 'D': 7, 'E': 9}
New best: 1 - {'A': 6, 'B': 4, 'C': 0, 'D': 2, 'E': 1}
Best found: 1 - {'A': 6, 'B': 4, 'C': 0, 'D': 2, 'E': 1}
```

Expected for country colors:

```
New best:  8  -  {'Loux': 'blue', 'Belgium': 'blue', 'Netherlands': 'blue', 'Germany': 'blue', 'France': 'blue'}
New best:  6  -  {'Loux': 'blue', 'Belgium': 'blue', 'Netherlands': 'red', 'Germany': 'blue', 'France': 'blue'}
New best:  4  -  {'Loux': 'blue', 'Belgium': 'red', 'Netherlands': 'blue', 'Germany': 'blue', 'France': 'blue'}
New best:  3  -  {'Loux': 'blue', 'Belgium': 'red', 'Netherlands': 'yellow', 'Germany': 'blue', 'France': 'blue'}
New best:  2  -  {'Loux': 'red', 'Belgium': 'red', 'Netherlands': 'yellow', 'Germany': 'blue', 'France': 'blue'}
New best:  1  -  {'Loux': 'red', 'Belgium': 'yellow', 'Netherlands': 'red', 'Germany': 'blue', 'France': 'blue'}
Best found: 1  -  {'Loux': 'red', 'Belgium': 'yellow', 'Netherlands': 'red', 'Germany': 'blue', 'France': 'blue'}
```